# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("output_data/cities.csv")
cities.head(1)

,Name,ID,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,lagoa,3372964.0,39.05,-27.98,62.94,86.0,100.0,32.86


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = cities[["Lat", "Lng"]]
humidity = cities["Humidity"]
maxHumidity = cities["Humidity"].max()

In [4]:
fig = gmaps.figure(center=(15,20), zoom_level=1.75)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=maxHumidity,
                                 point_radius = 2)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#I chose ideal the ideal city weather based on what I like, coldish and cloudy.

narrowed_city_df = cities.loc[((cities["Max Temp"]<40) & (cities["Max Temp"]>30)) & (cities["Wind Speed"]<=8) & \
                                (cities["Cloudiness"]>65)]
narrowed_city_df = narrowed_city_df.dropna()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame(columns = ["Hotel Name"])

hotel_df["Hotel Name"]= ""
hotel_df["Lat"] = ""
hotel_df["Lng"] = ""
hotel_df["City"] = ""
hotel_df["Country"]= ""
hotel_df["PlaceID"]=""

In [7]:
# #test for single call

# target_coordinates = "41.37, -73.41"
# target_search = "lodging"
# target_radius = 5000


# # set up a parameters dictionary
# params = {
#     "location": target_coordinates,
#     "keyword": target_search,
#     "radius": target_radius,
#     "key": g_key
# }

# # base url
# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# # run a request using our params dictionary
# response = requests.get(base_url, params=params)

# # convert response to json
# cityHotel = response.json()
# print(json.dumps(cityHotel, indent=4, sort_keys=True))

In [8]:
for index, row in narrowed_city_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    # geocoordinates
    target_coordinates = f"{lat}, {lng}"
    target_search = "lodging"
    target_radius = 5000


    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)

    # convert response to json
    cityHotel = response.json()
    try:
        hotel_df.loc[index, "Hotel Name"] = cityHotel["results"][0]["name"]
        hotel_df.loc[index, "Lat"] = cityHotel["results"][0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, "Lng"] = cityHotel["results"][0]["geometry"]["location"]["lng"]
        hotel_df.loc[index, "PlaceID"] = cityHotel["results"][0]["place_id"]
        #hotel_df.loc[index, "Vicinity"] = cityHotel["results"][0]["vicinity"]
    except:
        print("Ruh-Roh, no data. Skipping...")

Ruh-Roh, no data. Skipping...
Ruh-Roh, no data. Skipping...


In [9]:
hotel_df

,Hotel Name,Lat,Lng,City,Country,PlaceID
63,Chautauqua Harbor Hotel,42.1102,-79.2849,NaN,NaN,ChIJ6zM4eV6p0okRE5a0-lRuihI
155,"Kuroshio no Yado Rinkaisou, Minami-Boso Katsuu...",35.1506,140.308,NaN,NaN,ChIJrdlMc51JPWARpoYoFooN0mQ
181,Molino del Corregidor,42.2248,-2.48124,NaN,NaN,ChIJGQDTo8BKRQ0Re6ELUhFVpfU
198,CCI Express Inn,37.2427,-87.1499,NaN,NaN,ChIJ8__hEP5hZYgR0r2XThLM8IQ
248,Hotel New Palace,33.728,135.992,NaN,NaN,ChIJCV0SKLJsBmARPA6GSFzAXtM
256,Hotel Bhandari,30.73,78.437,NaN,NaN,ChIJ5fZTtCXtCDkRDhwpW7xS7dM
263,Siesta Motel,43.7023,-98.0103,NaN,NaN,ChIJSYhPy34RhocRn1vNhr3njlc
305,Pensiunea Sophia,46.4219,26.4404,NaN,NaN,ChIJL8vhdwc9tUARn6g11Toi8y4
307,Aspen Suites Hotel Sitka,57.0522,-135.333,NaN,NaN,ChIJcXnQf7tnBFQRb_b9obuKyro
318,Prestige Hudson Bay Lodge,54.7741,-127.156,NaN,NaN,ChIJ-RQyvQ8MdVQRc5KilyIF3pE


In [10]:
# #finding city name and country
for index, row in hotel_df.iterrows():
    placeid = row["PlaceID"]
    
    params = {
        "place_id":placeid,
        "key": g_key
    }
    
    base = "https://maps.googleapis.com/maps/api/place/details/json"
    
    responseCity = requests.get(base, params=params)
    
    places = responseCity.json()
    
    try:
        hotel_df.loc[index, "City"] = places["result"]["address_components"][2]["long_name"]
        hotel_df.loc[index, "Country"] = places["result"]["address_components"][5]["long_name"]
        #print(json.dumps(places, indent=4, sort_keys=True))
    except:
        print("Roh-Roh, no data. Skipping...")
        

Roh-Roh, no data. Skipping...
Roh-Roh, no data. Skipping...


In [11]:
# I realize that the above code doesn't quite get the city and country right as it varies in it's index for the city
# and country data. Below is what I attemped to do to make it work. I tried to loop through the indexes to find were
# places["result"]["address_component"]["type"=="country"] and then call that to get the country name, and then tried 
# numerous other things and I just couldn't wrap my head around how to make it work. I even tried playing around with 
# the autocomplete API to see if I could get better results from that, and I couldn't make it work.

#testing with loop

# placeid = "ChIJhRkd8Nw8I2ARdmilK03XEBI"

# params = {
#     "place_id":placeid,
#     "key": g_key
# }

# base = "https://maps.googleapis.com/maps/api/place/details/json"

# responseCity = requests.get(base, params=params)

# places = responseCity.json()

# i=0

# #try:
# while i <=5:
#     print(places["result"]["address_components"]["country"]["long_name"])
# #         print(places["result"]["address_components"]["types"]["long_name"])
# #         print("yay")
#     #print(places["result"]["address_components"][i]["types"])

#     #testdf.loc[index, "City"] = places["result"]["address_components"][i]["long_name"]
#     i+=1

#     #print(json.dumps(places, indent=4, sort_keys=True))

# # except:
# #     print("Roh-Roh, no data. Skipping...")

In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)


# Display Map

fig = gmaps.figure(center=(15,20), zoom_level=1.75)
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))